<a href="https://colab.research.google.com/github/llunasanz/data-project-3/blob/feature%2Fdata_processing/DataProject3LimpiezaDeDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyspark

     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 44.8 MB/s 
ERROR: Operation cancelled by user


In [ ]:
pip install sweetviz

     |█████████████████████▊          | 10.3 MB 5.2 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [1]:
import pandas as pd
#import pyspark as ps
import datetime as dt
#import sweetviz as sv
import matplotlib as plt
import datetime as dt

## CARGA Y LIMPIEZA DE DATOS

### Cargamos los datos de los diferentes CSV para su posterior limpieza de variables






In [65]:
# Usamos pandas para leer de los csv 
ts_perf = pd.read_csv('/content/drive/MyDrive/DP3/data/test_performance.csv')
ts_prev = pd.read_csv('/content/drive/MyDrive/DP3/data/test_previous_loan.csv')
tr_perf = pd.read_csv('/content/drive/MyDrive/DP3/data/train_performance.csv')
tr_prev = pd.read_csv('/content/drive/MyDrive/DP3/data/train_previous_loan.csv')
ts_demo = pd.read_csv('/content/drive/MyDrive/DP3/data/test_datos_demograficos.csv')
tr_demo = pd.read_csv('/content/drive/MyDrive/DP3/data/train_datos_demograficos.csv')

In [66]:
# Convertimos las cargas en dataframe

df_ts_performance = pd.DataFrame(ts_perf)
df_ts_previous = pd.DataFrame(ts_prev)
df_tr_performance = pd.DataFrame(tr_perf)
df_tr_previous = pd.DataFrame(tr_prev)
df_ts_demo = pd.DataFrame(ts_demo)
df_tr_demo = pd.DataFrame(tr_demo)

In [67]:
# Agrupamos los datasets en performance, previous y demographic

df_performance = pd.concat([df_tr_performance, df_tr_performance], axis = 0)
df_previous = pd.concat([df_tr_previous, df_tr_previous], axis = 0)
df_demo = pd.concat([df_ts_demo,df_tr_demo], axis = 0)


,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858f305c8dd672015c93b1db645db4,1976-08-28 00:00:00.000000,Savings,5.296628,7.593965,Heritage Bank,NaN,Permanent,NaN
1,8a858f085a477386015a47fb049e49ca,1978-06-23 00:00:00.000000,Savings,3.294513,6.596602,UBA,NaN,Permanent,NaN
2,8a858e6f5cd5e874015cd6f5634c39ad,1984-04-04 00:00:00.000000,Savings,8.501912,7.729364,First Bank,NaN,Permanent,NaN
3,8a858e9d5bfd7037015bfdab79f61305,1983-05-28 00:00:00.000000,Savings,3.318904,6.681595,UBA,NaN,Permanent,NaN
4,8a858fde56eb02280156eb6dafc128ac,1982-03-29 00:00:00.000000,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN
...,...,...,...,...,...,...,...,...,...
4341,8a858f155554552501555588ca2b3b40,1985-12-13 00:00:00.000000,Other,3.236753,7.030168,Stanbic IBTC,NaN,Permanent,Graduate
4342,8a858fc65cf978f4015cf97cee3a02ce,1982-07-01 00:00:00.000000,Savings,7.013750,4.875662,GT Bank,NaN,NaN,NaN
4343,8a858f4f5b66de3a015b66fc83c61902,1989-09-26 00:00:00.000000,Savings,6.295530,7.092508,GT Bank,NaN,Permanent,NaN
4344,8aaae7a74400b28201441c8b62514150,1985-09-06 00:00:00.000000,Savings,3.354206,6.539070,GT Bank,HEAD OFFICE,Permanent,Primary


In [68]:
# Añadimos identificador de performance

df_previous['performance'] = 0
df_performance['performance'] = 1

#### Limpieza del df previous




In [69]:
# Eliminamos la columna loannumber

df_previous = df_previous.drop(columns = 'systemloanid')

In [70]:
# Categorizamos la columna termdays para distinguir entre short, medium or long y quitamos la variable termdays

uv = df_previous['termdays'].unique()
print(uv)

df_previous['termlength'] = pd.cut(df_previous['termdays'],[0,16,61,89], labels = ['short','medium','large'])

#df_tr_performance.drop(columns = 'termdays')

[30 15 60 90]


In [71]:
# Categorizamos el referredby para que nos de 1 o 0

df_previous.referredby = list(map(lambda x: 0 if pd.isna(x) else 1, list(df_previous.referredby)))



In [72]:
# Modificamos el formato de la fecha del due date y el paid date para quitar las horas minutos y segundos

df_previous['firstduedate'] = pd.to_datetime(df_previous['firstduedate'], errors='coerce')
df_previous['firstduedate'] = df_previous['firstduedate'].dt.strftime('%d-%m-%Y')

df_previous['firstrepaiddate'] = pd.to_datetime(df_previous['firstrepaiddate'], errors='coerce')
df_previous['firstrepaiddate'] = df_previous['firstrepaiddate'].dt.strftime('%d-%m-%Y')

df_previous['closeddate'] = pd.to_datetime(df_previous['closeddate'], errors='coerce')
df_previous['closeddate'] = df_previous['closeddate'].dt.strftime('%d-%m-%Y')



In [73]:
# Como el approved date y el Creation date son el mismo, vamos a sacar la diferencia de horas para determinar si el tiempo para aceptarlo ha sido corto, medio o largo

df_previous['approveddate'] = pd.to_datetime(df_previous['approveddate'], errors='coerce')
df_previous['dateofaproval'] = df_previous['approveddate'].dt.strftime('%d-%m-%Y')
df_previous['approvedtime'] = df_previous['approveddate'].dt.strftime('%H:%M:%S')

df_previous['creationdate'] = pd.to_datetime(df_previous['creationdate'], errors='coerce')
df_previous['createdtime'] = df_previous['creationdate'].dt.strftime('%H:%M:%S')



In [74]:
# Como el approved date y el Creation date son el mismo, vamos sacar la diferencia de horas para determinar entre corto, medio o largo

df_previous['timediference'] = (df_previous['approveddate'] - df_previous['creationdate'])

# FALTEN COSES

In [90]:
df_performance.isnull().sum()

customerid       0
loannumber       0
loanamount       0
totaldue         0
termdays         0
referredby       0
good_bad_flag    0
performance      0
dtype: int64

In [76]:
# Vista del dataframe 
df_previous

,customerid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate,performance,termlength,dateofaproval,approvedtime,createdtime,timediference
0,8a2a81a74ce8c05d014cfb32a0da1049,2,2016-08-15 18:22:40,2016-08-15 17:22:32,10000.0,13000.0,30,01-09-2016,0,14-09-2016,01-09-2016,0,medium,15-08-2016,18:22:40,17:22:32,0 days 01:00:08
1,8a2a81a74ce8c05d014cfb32a0da1049,9,2017-04-28 18:39:07,2017-04-28 17:38:53,10000.0,13000.0,30,28-05-2017,0,30-05-2017,26-05-2017,0,medium,28-04-2017,18:39:07,17:38:53,0 days 01:00:14
2,8a2a81a74ce8c05d014cfb32a0da1049,8,2017-03-05 10:56:25,2017-03-05 09:56:19,20000.0,23800.0,30,26-04-2017,0,04-04-2017,26-04-2017,0,medium,05-03-2017,10:56:25,09:56:19,0 days 01:00:06
3,8a8588f35438fe12015444567666018e,5,2017-04-09 18:25:55,2017-04-09 17:25:42,10000.0,11500.0,15,24-04-2017,0,24-04-2017,24-04-2017,0,short,09-04-2017,18:25:55,17:25:42,0 days 01:00:13
4,8a85890754145ace015429211b513e16,2,2017-06-17 09:29:57,2017-06-17 08:29:50,10000.0,11500.0,15,14-07-2017,0,03-07-2017,14-07-2017,0,short,17-06-2017,09:29:57,08:29:50,0 days 01:00:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18178,8a858899538ddb8e0153a2b555421fc5,2,2016-04-16 13:36:34,2016-04-16 12:36:28,10000.0,13000.0,30,14-05-2016,0,16-05-2016,13-05-2016,0,medium,16-04-2016,13:36:34,12:36:28,0 days 01:00:06
18179,8a858899538ddb8e0153a2b555421fc5,9,2016-11-18 14:26:07,2016-11-18 13:25:51,30000.0,34400.0,30,13-12-2016,0,19-12-2016,13-12-2016,0,medium,18-11-2016,14:26:07,13:25:51,0 days 01:00:16
18180,8a858899538ddb8e0153a2b555421fc5,4,2016-06-12 15:30:56,2016-06-12 14:30:50,10000.0,13000.0,30,09-07-2016,0,12-07-2016,09-07-2016,0,medium,12-06-2016,15:30:56,14:30:50,0 days 01:00:06
18181,8a858f0656b7820c0156c92ca3ba436f,1,2016-08-27 20:03:45,2016-08-27 19:03:34,10000.0,13000.0,30,15-10-2016,0,26-09-2016,15-10-2016,0,medium,27-08-2016,20:03:45,19:03:34,0 days 01:00:11


#### Limpieza del df performance


In [77]:
#Eliminamos el systemloanid

df_performance = df_performance.drop(columns = 'systemloanid')

In [78]:
# Categorizamos el referredby para que nos de 1 o 0

df_performance.referredby = list(map(lambda x: 0 if pd.isna(x) else 1, list(df_performance.referredby)))

In [84]:
# Cambiamos el formato de las fechas

df_performance['approveddate'] = pd.to_datetime(df_ts_performance['approveddate'], exact=False , errors="coerce")
df_performance['creationdate'] = pd.to_datetime(df_ts_performance['creationdate'], exact=False , errors="coerce")

df_performance


,customerid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag,performance
0,8a2a81a74ce8c05d014cfb32a0da1049,12,NaT,NaT,30000.0,34500.0,30,0,Good,1
1,8a85886e54beabf90154c0a29ae757c0,2,NaT,NaT,15000.0,17250.0,30,0,Good,1
2,8a8588f35438fe12015444567666018e,7,2022-05-06 15:11:00,2022-05-06 15:04:00,20000.0,22250.0,15,0,Good,1
3,8a85890754145ace015429211b513e16,3,2022-05-06 00:54:00,2022-05-06 00:49:00,10000.0,11500.0,15,0,Good,1
4,8a858970548359cc0154883481981866,9,2022-05-06 04:33:00,2022-05-06 04:27:00,40000.0,44000.0,30,0,Good,1
...,...,...,...,...,...,...,...,...,...,...
4363,8a858e6d58b0cc520158beeb14b22a5a,2,NaT,NaT,10000.0,13000.0,30,0,Bad,1
4364,8a858ee85cf400f5015cf44ab1c42d5c,2,NaT,NaT,10000.0,13000.0,30,0,Bad,1
4365,8a858f365b2547f3015b284597147c94,3,NaT,NaT,10000.0,11500.0,15,0,Bad,1
4366,8a858f935ca09667015ca0ee3bc63f51,2,NaT,NaT,10000.0,13000.0,30,1,Bad,1


In [86]:
# Como vemos que hay varios nulos, analizamos cuantos hay en toda la hoja

print(df_performance['approveddate'].isna().sum())
print(df_performance['creationdate'].isna().sum())


df_performance.isnull().sum()

7542
7554


customerid          0
loannumber          0
approveddate     7542
creationdate     7554
loanamount          0
totaldue            0
termdays            0
referredby          0
good_bad_flag       0
performance         0
dtype: int64

In [88]:
# Vemos que hay un total de 7566 nulos sobre las 8736 filas, por tanto eliminamos estas variables porque no van a aportarnos nada

df_performance = df_performance.drop(['approveddate','creationdate'], axis = 1)


In [114]:
# MODIFICAMOS EL FORMATO DE LA FECHA Y RESTAMOS LA FECHA ACTUAL PARA SABER LA EDAD DE ESTAS PERSONAS




#FALTAA!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [112]:
# Vista del dataframe 
df_performance

,customerid,loannumber,loanamount,totaldue,termdays,referredby,good_bad_flag,performance
0,8a2a81a74ce8c05d014cfb32a0da1049,12,30000.0,34500.0,30,0,Good,1
1,8a85886e54beabf90154c0a29ae757c0,2,15000.0,17250.0,30,0,Good,1
2,8a8588f35438fe12015444567666018e,7,20000.0,22250.0,15,0,Good,1
3,8a85890754145ace015429211b513e16,3,10000.0,11500.0,15,0,Good,1
4,8a858970548359cc0154883481981866,9,40000.0,44000.0,30,0,Good,1
...,...,...,...,...,...,...,...,...
4363,8a858e6d58b0cc520158beeb14b22a5a,2,10000.0,13000.0,30,0,Bad,1
4364,8a858ee85cf400f5015cf44ab1c42d5c,2,10000.0,13000.0,30,0,Bad,1
4365,8a858f365b2547f3015b284597147c94,3,10000.0,11500.0,15,0,Bad,1
4366,8a858f935ca09667015ca0ee3bc63f51,2,10000.0,13000.0,30,1,Bad,1


#### Limpieza Demographic


In [113]:
# Comprobamos la cantidad de nulos que tenemos, y vemos que tanto en la branch como el level of education no podremos sacar información

df_demo.isnull().sum()

customerid                     0
birthdate                      0
bank_account_type              0
longitude_gps                  0
latitude_gps                   0
bank_name_clients              0
employment_status_clients    865
dtype: int64

In [109]:
#Vamos a eliminar las dos columnas comentadas anteriormente porque no nos aportan información suficiente

df_demo = df_demo.drop(['bank_branch_clients','level_of_education_clients'], axis = 1)

In [110]:
# Vista del df Demographic

df_demo


,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients
0,8a858f305c8dd672015c93b1db645db4,1976-08-28 00:00:00.000000,Savings,5.296628,7.593965,Heritage Bank,Permanent
1,8a858f085a477386015a47fb049e49ca,1978-06-23 00:00:00.000000,Savings,3.294513,6.596602,UBA,Permanent
2,8a858e6f5cd5e874015cd6f5634c39ad,1984-04-04 00:00:00.000000,Savings,8.501912,7.729364,First Bank,Permanent
3,8a858e9d5bfd7037015bfdab79f61305,1983-05-28 00:00:00.000000,Savings,3.318904,6.681595,UBA,Permanent
4,8a858fde56eb02280156eb6dafc128ac,1982-03-29 00:00:00.000000,Savings,6.354624,4.949031,First Bank,Self-Employed
...,...,...,...,...,...,...,...
4341,8a858f155554552501555588ca2b3b40,1985-12-13 00:00:00.000000,Other,3.236753,7.030168,Stanbic IBTC,Permanent
4342,8a858fc65cf978f4015cf97cee3a02ce,1982-07-01 00:00:00.000000,Savings,7.013750,4.875662,GT Bank,NaN
4343,8a858f4f5b66de3a015b66fc83c61902,1989-09-26 00:00:00.000000,Savings,6.295530,7.092508,GT Bank,Permanent
4344,8aaae7a74400b28201441c8b62514150,1985-09-06 00:00:00.000000,Savings,3.354206,6.539070,GT Bank,Permanent


#### Concatenamos las dos tablas una vez están limpias

In [96]:
# Creamos un df concatenando las tablas de performance y previous y mergeamos la tabla de demographic en el customer id

df_modelo = pd.concat([df_performance, df_previous], axis = 0)
df_modelo = pd.merge(df_modelo, df_demo, on='customerid')
df_modelo

,customerid,loannumber,loanamount,totaldue,termdays,referredby,good_bad_flag,performance,approveddate,creationdate,...,createdtime,timediference,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a2a81a74ce8c05d014cfb32a0da1049,12,30000.0,34500.0,30,0,Good,1,NaT,NaT,...,NaN,NaT,1972-01-15 00:00:00.000000,Other,3.432010,6.433055,Diamond Bank,NaN,Permanent,Post-Graduate
1,8a2a81a74ce8c05d014cfb32a0da1049,12,30000.0,34500.0,30,0,Good,1,NaT,NaT,...,NaN,NaT,1972-01-15 00:00:00.000000,Other,3.432010,6.433055,Diamond Bank,NaN,Permanent,Post-Graduate
2,8a2a81a74ce8c05d014cfb32a0da1049,2,10000.0,13000.0,30,0,NaN,0,2016-08-15 18:22:40,2016-08-15 17:22:32,...,17:22:32,0 days 01:00:08,1972-01-15 00:00:00.000000,Other,3.432010,6.433055,Diamond Bank,NaN,Permanent,Post-Graduate
3,8a2a81a74ce8c05d014cfb32a0da1049,9,10000.0,13000.0,30,0,NaN,0,2017-04-28 18:39:07,2017-04-28 17:38:53,...,17:38:53,0 days 01:00:14,1972-01-15 00:00:00.000000,Other,3.432010,6.433055,Diamond Bank,NaN,Permanent,Post-Graduate
4,8a2a81a74ce8c05d014cfb32a0da1049,8,20000.0,23800.0,30,0,NaN,0,2017-03-05 10:56:25,2017-03-05 09:56:19,...,09:56:19,0 days 01:00:06,1972-01-15 00:00:00.000000,Other,3.432010,6.433055,Diamond Bank,NaN,Permanent,Post-Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45179,8a858fd458639fcc015868eb14b542ad,1,10000.0,13000.0,30,0,NaN,0,2016-11-15 20:35:26,2016-11-15 19:35:10,...,19:35:10,0 days 01:00:16,1988-08-16 00:00:00.000000,Savings,3.725655,6.474430,GT Bank,NaN,NaN,NaN
45180,8a858fd458639fcc015868eb14b542ad,4,20000.0,23800.0,30,0,NaN,0,2017-02-12 23:18:28,2017-02-12 22:18:22,...,22:18:22,0 days 01:00:06,1988-08-16 00:00:00.000000,Savings,3.725655,6.474430,GT Bank,NaN,NaN,NaN
45181,8a858fd458639fcc015868eb14b542ad,5,20000.0,23800.0,30,0,NaN,0,2017-04-12 18:54:51,2017-04-12 17:54:39,...,17:54:39,0 days 01:00:12,1988-08-16 00:00:00.000000,Savings,3.725655,6.474430,GT Bank,NaN,NaN,NaN
45182,8a858fd458639fcc015868eb14b542ad,2,10000.0,13000.0,30,0,NaN,0,2016-12-12 23:54:06,2016-12-12 22:53:56,...,22:53:56,0 days 01:00:10,1988-08-16 00:00:00.000000,Savings,3.725655,6.474430,GT Bank,NaN,NaN,NaN


Lo descargamos en csv

In [18]:
train_demografic.to_csv('df_demographic.csv', index = False , sep = ',')

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Ahora queremos mergear la de demografic con el performance para sacar la edad


In [ ]:
prev = pd.read_csv('/content/df_previous.csv')

In [ ]:
demo = pd.read_csv('/content/df_demographic.csv')

In [ ]:
demo.head()

In [ ]:
df_prev_demo = pd.merge(demo, prev, on='customerid')

In [ ]:
df_prev_demo.head()